# Определение стоимости автомобилей

Сервис по продаже автомобилей с пробегом «Не бит, не крашен» разрабатывает приложение для привлечения новых клиентов. В нём можно быстро узнать рыночную стоимость своего автомобиля. В вашем распоряжении исторические данные: технические характеристики, комплектации и цены автомобилей. Вам нужно построить модель для определения стоимости. 

Заказчику важны:

- качество предсказания;
- скорость предсказания;
- время обучения.

In [ ]:
import pandas as pd
import numpy as np

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import seaborn as sns

import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.linear_model import LinearRegression 
from sklearn.ensemble import RandomForestRegressor

from catboost import CatBoostRegressor

from lightgbm import LGBMRegressor

import warnings

warnings.filterwarnings('ignore')

## Подготовка данных

In [ ]:
try:

    df_authentic = pd.read_csv('/Users/admin/Desktop/Rudra PRACTICUM/Projects/my_project/Datasets/autos.csv')
    
except:
    
    df_authentic = pd.read_csv('/datasets/autos.csv')

In [ ]:
df_authentic.head()

In [ ]:
#обратил внимание на название столбцов и решил их перевести в рабочий вид

df_authentic.set_axis(['date_crawled', 
            'price', 
            'vehicle_type',
            'registration_year', 
            'gearbox', 
            'power', 
            'model', 
            'kilometer', 
            'registration_month', 
            'fuel_type', 
            'brand', 
            'not_repaired', 
            'date_created', 
            'number_of_pictures', 
            'postal_code', 
            'last_seen'], axis='columns', inplace=True)

In [ ]:
df_authentic.info()

In [ ]:
df_authentic.isnull().sum()

In [ ]:
# удалили на мой взгляд ненужные столбцы

df_authentic.drop(
    ['date_crawled', 'date_created', 'number_of_pictures', 'postal_code', 'last_seen'], 
    1, inplace=True
)

### Визуализирую данные

In [ ]:
df_authentic.corr()

In [ ]:
_, axs = plt.subplots(4, 1, figsize=(17, 20))

axs[0].bar(
    df_authentic.vehicle_type.fillna('missed').value_counts().index, 
    df_authentic.vehicle_type.fillna('missed').value_counts()
)
axs[1].bar(
    df_authentic.gearbox.fillna('missed').value_counts(dropna=False).index, 
    df_authentic.gearbox.fillna('missed').value_counts(dropna=False)
)
axs[2].bar(
    df_authentic.fuel_type.fillna('missed').value_counts(dropna=False).index, 
    df_authentic.fuel_type.fillna('missed').value_counts(dropna=False)
)
axs[3].bar(
    df_authentic.not_repaired.fillna('missed').value_counts(dropna=False).index, 
    df_authentic.not_repaired.fillna('missed').value_counts(dropna=False)
)
axs[0].set_title('vehicle_type')
axs[1].set_title('gearbox')
axs[2].set_title('fuel_type')
axs[3].set_title('not_repaired')
plt.show()

In [ ]:
categorical_columns = [c for c in df_authentic.columns if df_authentic[c].dtype.name == 'object']

df_authentic[categorical_columns].describe()

In [ ]:
sns.set_style('whitegrid')

_, axs = plt.subplots(1, 4, figsize=(17, 7))

axs[0].boxplot(x=df_authentic['price'])
axs[1].boxplot(x=df_authentic['registration_year'])
axs[2].boxplot(x=df_authentic['power'])
axs[3].boxplot(x=df_authentic['kilometer'])

axs[0].set_title('price')
axs[1].set_title('registration_year')
axs[2].set_title('power')
axs[3].set_title('kilometer')

axs[1].set_ylim(1980, 2025)
axs[2].set_ylim(0, 1000)

plt.show()

In [ ]:
df_authentic.describe()

### Заполняю пропуска

In [ ]:
# заполнил пропущенные значения в столбце vehicle_type, gearbox, model, fuel_type, not_repaired
# так как считаю что пользователи просто непрописывали данный признак

df_authentic[
    ['vehicle_type', 'gearbox', 'model', 'fuel_type', 'not_repaired']
] = df_authentic[
    ['vehicle_type', 'gearbox', 'model', 'fuel_type', 'not_repaired']
].fillna('missed')

### Проверяю выбросы

In [ ]:
# Заменил все на мой взгляд неодыкватные значения столбца registration_year на 0

df_authentic.loc[
    (df_authentic.registration_year < 1986) | (df_authentic.registration_year > 2019), 
    'registration_year'
] = 0

In [ ]:
# Заменил все на мой взгляд неодыкватные значения столбца power на 0  

df_authentic.loc[
    df_authentic.power > 1000, 
    'power'
] = 0

# Перехожу к построению моделей

In [ ]:
# изменил название для удобства

df = df_authentic

In [ ]:
y = df.price

In [ ]:
# кодирование с помощь метода OHE для линейных моделей

X_ohe = pd.get_dummies(
    df.drop(['price'], 1),
    drop_first=True
)

In [ ]:
categorical_features = ['vehicle_type', 'gearbox', 'model', 'fuel_type', 'brand', 'not_repaired']
encoder = OrdinalEncoder()
encoder.fit(df[categorical_features])
df[categorical_features] = encoder.transform(df[categorical_features])
X_encoder = df.drop('price', 1)

In [ ]:
# для линейных моделей 

X_ohe_train, X_ohe_test, y_ohe_train, y_ohe_test = train_test_split(
    X_ohe, y, test_size=0.2, random_state=12345
)

X_ohe_train, X_ohe_valid, y_ohe_train, y_ohe_valid = train_test_split(
    X_ohe_train, y_ohe_train, test_size=0.25, random_state=12345
)

In [ ]:
# для леса

X_encoder_train, X_encoder_test, y_encoder_train, y_encoder_test = train_test_split(
    X_encoder, y, test_size=0.2, random_state=12345
)

X_encoder_train, X_encoder_valid, y_encoder_train, y_encoder_valid = train_test_split(
    X_encoder_train, y_encoder_train, test_size=0.25, random_state=12345
)

In [ ]:
# проведу стандартизацию

numeric = ['power', 'kilometer', 'registration_month', 'registration_year']

scaler_ohe = StandardScaler()
scaler_encoder = StandardScaler()
scaler_ohe.fit(X_ohe_train[numeric])
scaler_encoder.fit(X_encoder_train[numeric])

X_ohe_train[numeric] = scaler_ohe.transform(X_ohe_train[numeric])
X_ohe_valid[numeric] = scaler_ohe.transform(X_ohe_valid[numeric]) 
X_ohe_test[numeric] = scaler_ohe.transform(X_ohe_test[numeric])

X_encoder_train[numeric] = scaler_encoder.transform(X_encoder_train[numeric])
X_encoder_valid[numeric] = scaler_encoder.transform(X_encoder_valid[numeric]) 
X_encoder_test[numeric] = scaler_encoder.transform(X_encoder_test[numeric])
pd.options.mode.chained_assignment = None

## Обучение моделей

In [ ]:
parametrs_rfr = {
    'max_depth': [3, 4, 5], 
    'n_estimators': [30, 40, 50]
}

In [ ]:
def ML(model_ml, features_train, target_train, features_valid, target_valid, parametrs, **fiter):
    clf = model_ml()
    grid = GridSearchCV(clf, parametrs, cv=5, scoring='neg_mean_squared_error')
    grid.fit(features_train, target_train)
    g = grid.best_params_
    
    model = model_ml(random_state=12345, **g)
    model.fit(features_train, target_train, **fiter)
    predicted_valid = model.predict(features_valid)
    mse = mean_squared_error(target_valid, predicted_valid)
    rmse = mse ** 0.5
    print(f'RMSE = {rmse}, гиперпараметры {g}')
    
    return model

In [ ]:
parametrs_cb = {
    'loss_function': ['RMSE'],
    'iterations': [1000],
    'depth': [15],
    'verbose': [False]
}

In [ ]:
%%time

CatBoostRegressor_model = ML(
     CatBoostRegressor, 
     X_encoder_train, 
     y_encoder_train, 
     X_encoder_valid, 
     y_encoder_valid, 
     parametrs_cb, 
     verbose=False
);

## Линейная регрессия

In [ ]:
model_linearregression = LinearRegression()

In [ ]:
%%time
model_linearregression.fit(X_ohe_train, y_ohe_train);

In [ ]:
predict_LR = model_linearregression.predict(X_ohe_valid)

In [ ]:
mse = mean_squared_error(y_ohe_valid, predict_LR)
rmse = mse ** 0.5

In [ ]:
print(f'RMSE {rmse}')

## Случайный лес

In [ ]:
parametrs_rfr = {
    'max_depth': [15], 
    'n_estimators': [100]
}

In [ ]:
%%time
RandomForestRegressor_model = ML(
    RandomForestRegressor, X_encoder_train, y_encoder_train, X_encoder_valid, y_encoder_valid, parametrs_rfr
)

## LightGBM

In [ ]:
parametrs_LGBM = {
    'max_depth': [15], 
    'n_estimators': [1000]
}

In [ ]:
%%time
LGBMRegressor_model = ML(
    LGBMRegressor, X_encoder_train, y_encoder_train, X_encoder_valid, y_encoder_valid, parametrs_LGBM
)

## Проверяю время на тестовой выбоке

# CatBoost

In [ ]:
%%time
pred = CatBoostRegressor_model.predict(X_encoder_test)
mse = mean_squared_error(y_encoder_test, pred)
rmse = mse ** 0.5
print(f'RMSE {rmse}')

# LinearRegression

In [ ]:
%%time
predict_LR = model_linearregression.predict(X_ohe_test)
mse = mean_squared_error(y_ohe_test, predict_LR)
rmse = mse ** 0.5
print(f'RMSE {rmse}')

# RandomForestRegressor

In [ ]:
%%time
predicted_test = RandomForestRegressor_model[1].predict(X_encoder_test)
mse = mean_squared_error(y_encoder_test, predicted_test)
rmse = mse ** 0.5
print(f'RMSE {rmse}')

# LGBMRegressor

In [ ]:
%%time
predicted_test = LGBMRegressor_model.predict(X_encoder_test)
mse = mean_squared_error(y_encoder_test, predicted_test)
rmse = mse ** 0.5
print(f'RMSE {rmse}')

# Вывод

самой быстрой, но самой худшей по предсказаниям оказалась модель LinearRegression
RandomForestRegressor оказался очень долгим
CatBoost результаты неплозие, но долгое обучение
LGBMRegressor отличный результат и маленькое время обучения